# Assignment 7

## Submit as an HTML file

### Print your name below

In [2]:
print("Winnie Lau")

Winnie Lau


### Import the "pandas" "numpy" and "statsmodels.formula.api" libraries

In [3]:
# Write your answer here:

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

#### In the code chunk below read the CSV file named `results.csv` in the `data` <br> folder and print the first 5 rows of the dataset. Browse the dataset.

In [4]:
results = pd.read_csv("data/results.csv")
results.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


### (a)  Check Column Types and Data Cleaning

- Use the function .dtypes to get the column types
- Identify which columns have data types that might need conversion
- The 'milliseconds' column contains string values that should be numeric. Create a new column called 'race_time_ms' that:
    - Converts the column to a numeric data type
    - Replaces any non-numeric values with NaN

In [5]:
# Write your answer here

results.dtypes

results["milliseconds"].str.isnumeric()
subset = results.query("milliseconds.str.isnumeric()==False")
list_unique = pd.unique(subset["milliseconds"])
list_unique


results["race_time_ms"] = results["milliseconds"].replace("\\N", np.nan)
results["race_time_ms"] = pd.to_numeric(results["race_time_ms"])


### (b) Create Categorical Variables

- Create a new column called 'finish_category' that categorizes the race finish positions as follows:
    - Positions 1-3: 'Podium'
    - Positions 4-10: 'Points'
    - Positions 11-20: 'Midfield'
    - Positions >20: 'Backmarker'

Hint: Use the pd.cut() function

In [15]:
# Write your answer here

results["position"] = results["position"].replace("\\N", np.nan)
results["position"] = pd.to_numeric(results["position"])

bins1 = [0,3,10,20,40]
labels1 = ["Podium", "Points", "Midfield", "Backmarker"]

results["finish_category"] = pd.cut(results["position"],
                                    bins = bins1,
                                    right = True,
                                    labels = labels1)


### (c) Calculate Race Duration
- For rows where 'milliseconds' is available, create a new column <br>
'race_duration_minutes' that converts milliseconds to minutes by dividing <br>
by (1000*60).
- Display the average race duration by 'constructorId' for the top 5 <br>
constructors with the shortest average race times

In [7]:
# Write your answer here

results["race_duration_minutes"] = results["race_time_ms"]/(1000*60)

duration_agg = (results.groupby("constructorId")
                .agg(avg_race_duration = ("race_duration_minutes", "mean"))
                .sort_values(by="avg_race_duration", ascending=True))

duration_agg.head()

,avg_race_duration
constructorId,
35,76.710777
29,77.604125
41,87.046767
16,89.428828
53,89.658852


### (d) Driver Performance Analysis

- Calculate the following statistics for each driver, grouped by 'driverId':
    - Average finishing position
    - Total points
    - Number of races completed
    - Best finishing position

- Sort the results by total points in descending order
- Display the top 10 drivers based on total points

In [25]:
# Write your answer here

driver_agg = (results.groupby("driverId")
              .agg(avg_finish_pos = ("position", "mean"),
                   total_points = ("points", "sum"),
                   num_race_compl = ("driverId", len),
                   best_fin_pos = ("finish_category", "min"))
                   .sort_values(by="total_points", ascending=False))

driver_agg.head(10)


,avg_finish_pos,total_points,num_race_compl,best_fin_pos
driverId,,,,
1,3.383803,4396.5,310,Podium
20,5.282443,3098.0,300,Podium
4,6.257732,2061.0,358,Podium
830,4.022388,1983.5,163,Podium
8,6.238596,1873.0,352,Podium
822,6.296089,1778.0,201,Podium
3,6.355932,1594.5,206,Podium
30,3.701245,1566.0,308,Podium
817,8.261538,1307.0,232,Podium


### (e) Linear Regression
Create a linear regression model that predicts 'points' based on 'grid' (starting position) and 'laps' completed <br>
Use the following steps:

- Clean the data to remove any non-numeric values and missing values
- Create the regression formula using smf.ols 
- Display the summary of the regression model using model.summary()

What is the predicted points for a driver starting in position 3 and completing 55 laps?

Hint: Use ```.dropna()''' to remove missing values from the points, grid, and laps <br>
variables.

In [30]:
# Write your answer here

list1 = ["points", "grid", "laps"]
for i in list1:
    results[i].dropna()

model = smf.ols(formula = 'points ~ grid+laps', data = results)
outcome = model.fit()
display(outcome.summary())

b_list = outcome.params
print(b_list)

pred_points = b_list[0] + b_list[1]*(3) + b_list[2]*(55)
print("The predicted points for a driver starting in position 3 and completeing 55 laps is", pred_points)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 points   R-squared:                       0.215
Model:                            OLS   Adj. R-squared:                  0.215
Method:                 Least Squares   F-statistic:                     3530.
Date:                Mon, 24 Mar 2025   Prob (F-statistic):               0.00
Time:                        13:34:10   Log-Likelihood:                -70440.
No. Observations:               25840   AIC:                         1.409e+05
Df Residuals:                   25837   BIC:                         1.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.5841      0.054     48.267      0.000       2.479       2.689
grid          -0.2248      0.003    -70.621      0.000      -0.231      -0.219
laps           0.0393      0.001     50.779      0.000       0.038       0.041
==============================================================================
Omnibus:                    15818.159   Durbin-Watson:                   0.715
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           183648.109
Skew:                           2.804   Prob(JB):                         0.00
Kurtosis:                      14.795   Cond. No.                         130.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Intercept    2.584126
grid        -0.224796
laps         0.039279
dtype: float64
The predicted points for a driver starting in position 3 and completeing 55 laps is 4.070062614810155
